In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 필요한 모듈 생성

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 데이터 불러오기

In [3]:
train = pd.read_csv('/content/drive/MyDrive/2022-2 여름방학 와이빅타/fe/train.csv')
train = train.drop('id', axis=1)

In [4]:
print(train.shape)
train.head()

(17480, 15)


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0


# Feature Engineering

## 결측치 확인

In [5]:
# 두 코드 모두 가능!
train.info()
train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17480 entries, 0 to 17479
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             17480 non-null  int64 
 1   workclass       15644 non-null  object
 2   fnlwgt          17480 non-null  int64 
 3   education       17480 non-null  object
 4   education.num   17480 non-null  int64 
 5   marital.status  17480 non-null  object
 6   occupation      15637 non-null  object
 7   relationship    17480 non-null  object
 8   race            17480 non-null  object
 9   sex             17480 non-null  object
 10  capital.gain    17480 non-null  int64 
 11  capital.loss    17480 non-null  int64 
 12  hours.per.week  17480 non-null  int64 
 13  native.country  16897 non-null  object
 14  target          17480 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 2.0+ MB


age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
target               0
dtype: int64

## 결측치 제거

In [6]:
train_1 = train.dropna()
print("결측치 제거 전 데이터 개수 : ", train.shape[0])
print("결측치 제거 후 데이터 개수 : ", train_1.shape[0])

결측치 제거 전 데이터 개수 :  17480
결측치 제거 후 데이터 개수 :  15081


In [7]:
# 결측치가 모두 사라진것을 볼 수 있음
train_1.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
target            0
dtype: int64

## 결측치 대체

In [8]:
####결측치를 0으로 대체####
train_2 = train.fillna(0)
print("결측치 대체 전 데이터 개수 : ", train.shape[0])
print("결측치 대체 후 데이터 개수 : ", train_2.shape[0])

결측치 대체 전 데이터 개수 :  17480
결측치 대체 후 데이터 개수 :  17480


- 결측치를 0으로 대체해줌으로써 데이터 개수는 기존 데이터와 같은것을 볼 수 있음

In [9]:
####결측치를 Mean or Median으로 대체####
train_3 = train.fillna(train.mean())
train_4 = train.fillna(train.median())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
####결측치를 가장 빈도수가 높은 값으로 대체####
print(train['workclass'].value_counts())
print(train['occupation'].value_counts())
print(train['native.country'].value_counts())

Private             11568
Self-emp-not-inc     1272
Local-gov            1053
State-gov             659
Self-emp-inc          594
Federal-gov           485
Never-worked            7
Without-pay             6
Name: workclass, dtype: int64
Exec-managerial      2113
Craft-repair         2101
Prof-specialty       2085
Adm-clerical         1893
Sales                1829
Other-service        1677
Machine-op-inspct    1040
Transport-moving      785
Handlers-cleaners     695
Farming-fishing       508
Tech-support          475
Protective-serv       350
Priv-house-serv        83
Armed-Forces            3
Name: occupation, dtype: int64
United-States                 15393
Mexico                          355
Philippines                     108
Germany                          75
Canada                           63
Puerto-Rico                      59
El-Salvador                      58
Cuba                             49
India                            47
England                          46
Jamaica

- 일단 결측치가 존재하는 Feature에서 어떤 unique값이 가장 빈도수가 높은지 찾아봄
    - workclass : Private
    - occupation : Exec-managerial
    - native.country : United-States

In [11]:
# 빈도수가 높은 값으로 채워줌
train_5 = train.copy()
train_5.loc[train['workclass'] != train['workclass'],'workclass'] = 'Private'
train_5.loc[train['occupation'] != train['occupation'],'occupation'] = 'Exec-managerial'
train_5.loc[train['native.country'] != train['native.country'],'native.country'] = 'United-States'
print("결측값 대체 전 workclass의 Private 값 개수 : ", train.workclass.value_counts()[0])
print("결측값 대체 후 workclass의 Private 값 개수 : ", train_5.workclass.value_counts()[0])

결측값 대체 전 workclass의 Private 값 개수 :  11568
결측값 대체 후 workclass의 Private 값 개수 :  13404


- 결측치를 빈도수가 가장 높았던 Private으로 대체해줌으로써, Private값의 개수가 늘었음을 확인할 수 있음

In [12]:
####결측치를 임의의 값으로 대체####
train_6 = train.fillna(method = 'pad')
train_7 = train.fillna(method = 'bfill')

- pad : 특정한 수치를사용하지 않고, 결측치의 바로 앞에 있는 value를 채워넣는 방식
- bfill : pad와는 반대로 각 결측치의 바로 뒤의 값으로 채워넣는 방법

# 지표변수 (Indicator variables)

In [13]:
train['age']

0        32
1        33
2        46
3        23
4        55
         ..
17475    35
17476    30
17477    71
17478    41
17479    72
Name: age, Length: 17480, dtype: int64

In [14]:
oldness = []
for a in train['age']:
    if (17 <= a <= 30):
        oldness.append('young')
    elif (31 <= a <= 50):
        oldness.append('middle')
    else:
        oldness.append('old')

train['oldness'] = oldness

In [15]:
train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old


## Feature Split

In [16]:
train['date'] = '2022-02-03'
train

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness,date
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle,2022-02-03
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young,2022-02-03
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old,2022-02-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17475,35,NaN,320084,Bachelors,13,Married-civ-spouse,NaN,Wife,White,Female,0,0,55,United-States,1,middle,2022-02-03
17476,30,NaN,33811,Bachelors,13,Never-married,NaN,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,0,young,2022-02-03
17477,71,NaN,287372,Doctorate,16,Married-civ-spouse,NaN,Husband,White,Male,0,0,10,United-States,1,old,2022-02-03
17478,41,NaN,202822,HS-grad,9,Separated,NaN,Not-in-family,Black,Female,0,0,32,United-States,0,middle,2022-02-03


In [17]:
year = []
month = []
day = []

for d in train['date'].str.split("-"):
    year.append(d[0])
    month.append(d[1])
    day.append(d[2])
train['year'] = year
train['month'] = month
train['day'] = day

In [18]:
train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness,date,year,month,day
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03,2022,02,03
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle,2022-02-03,2022,02,03
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03,2022,02,03
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young,2022-02-03,2022,02,03
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old,2022-02-03,2022,02,03


## 스케일링 Scaling

In [19]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
mm_scaler = MinMaxScaler()
sd_scaler = StandardScaler()

In [20]:
# MinMax Scaling
mm_scaler = mm_scaler.fit_transform(train['age'].values.reshape(-1,1))
train['mm_scaled_age'] = mm_scaler

# Standard Scaling
sd_scaler = sd_scaler.fit_transform(train['age'].values.reshape(-1,1))
train['sd_scaled_age'] = sd_scaler


In [21]:
train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,...,hours.per.week,native.country,target,oldness,date,year,month,day,mm_scaled_age,sd_scaled_age
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,0,middle,2022-02-03,2022,02,03,0.205479,-0.477370
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,40,United-States,1,middle,2022-02-03,2022,02,03,0.219178,-0.406343
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,0,middle,2022-02-03,2022,02,03,0.397260,0.517004
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,...,30,United-States,0,young,2022-02-03,2022,02,03,0.082192,-1.116610
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,...,40,United-States,0,old,2022-02-03,2022,02,03,0.520548,1.156244


## 라벨 인코딩 (Label Encoding)

In [22]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [23]:
encoded = encoder.fit_transform(train['education'])

In [24]:
train['education_class'] = encoded

In [25]:
train.education_class.value_counts()

11    5566
15    3905
9     2842
12     921
8      724
1      664
7      588
0      510
5      378
14     301
6      294
2      249
10     214
4      197
3       96
13      31
Name: education_class, dtype: int64

## 원-핫 인코딩 (One-Hot Encoding)

In [26]:
pd.get_dummies(train['education'])

,10th,11th,12th,1st-4th,5th-6th,7th-8th,9th,Assoc-acdm,Assoc-voc,Bachelors,Doctorate,HS-grad,Masters,Preschool,Prof-school,Some-college
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
17476,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
17477,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
17478,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


#과제



```
# 코드로 형식 지정됨
```

##Titanic 데이터를 가지고 5 개의 FE 방법들 직접 사용해보기 (오늘 피피티에 소개되지 않은 FE 방법도 괜찮음)

데이터 불러오기

In [27]:
titanic = pd.read_csv('/content/drive/MyDrive/2022-2 여름방학 와이빅타/fe/titanic.csv')
titanic.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [29]:
titanic.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

1. Imputation

In [32]:
titanic1 = titanic.fillna(titanic.median())
titanic1.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [33]:
titanic1.isnull().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [34]:
titanic1 = titanic1.fillna("DEFAULT")
titanic1.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,DEFAULT,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,DEFAULT,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,DEFAULT,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,DEFAULT,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,DEFAULT,S


2. Indicator Variables

In [36]:
oldness = []
for a in titanic['Age']:
    if (a <= 30):
        oldness.append('young')
    elif (30 < a <= 50):
        oldness.append('middle')
    else:
        oldness.append('old')

titanic2 = titanic
titanic2['oldness'] = oldness

titanic2.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,oldness
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,middle
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,middle
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,old
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,young
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,young


3. Feature Split

In [37]:
titanic3 = titanic
first_name = []
last_name = []

for n in titanic['Name'].str.split(","):
  first_name.append(n[0])
  last_name.append(n[1])

titanic3['first_name'] = first_name
titanic3['last_name'] = last_name
titanic3.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,oldness,first_name,last_name
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,middle,Kelly,Mr. James
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,middle,Wilkes,Mrs. James (Ellen Needs)
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,old,Myles,Mr. Thomas Francis
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,young,Wirz,Mr. Albert
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,young,Hirvonen,Mrs. Alexander (Helga E Lindqvist)


4. Scaling

In [38]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
mm_scaler = MinMaxScaler()
sd_scaler = StandardScaler()

# MinMax Scaling
mm_scaler = mm_scaler.fit_transform(titanic['Age'].values.reshape(-1,1))
titanic['mm_scaled_age'] = mm_scaler

# Standard Scaling
sd_scaler = sd_scaler.fit_transform(titanic['Age'].values.reshape(-1,1))
titanic['sd_scaled_age'] = sd_scaler

train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,...,native.country,target,oldness,date,year,month,day,mm_scaled_age,sd_scaled_age,education_class
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,...,United-States,0,middle,2022-02-03,2022,02,03,0.205479,-0.477370,7
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,United-States,1,middle,2022-02-03,2022,02,03,0.219178,-0.406343,15
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,...,United-States,0,middle,2022-02-03,2022,02,03,0.397260,0.517004,15
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,...,United-States,0,young,2022-02-03,2022,02,03,0.082192,-1.116610,9
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,...,United-States,0,old,2022-02-03,2022,02,03,0.520548,1.156244,11


5. One-Hot Encoding

In [41]:
titanic5 = titanic
pd.get_dummies(titanic5['Age']).head()

,0.17,0.33,0.75,0.83,0.92,1.00,2.00,3.00,5.00,6.00,...,58.00,59.00,60.00,60.50,61.00,62.00,63.00,64.00,67.00,76.00
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
pd.get_dummies(titanic5['Fare']).head()

,0.0000,3.1708,6.4375,6.4958,6.9500,7.0000,7.0500,7.2250,7.2292,7.2500,...,151.5500,164.8667,211.3375,211.5000,221.7792,227.5250,247.5208,262.3750,263.0000,512.3292
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
